Pinned Documentary URL <br>
* [Diffusers Github](https://github.com/huggingface/diffusers)
* [Diffusers HF](https://huggingface.co/docs/diffusers/index)
* [WdangTehSeeker3245 Github](https://github.com/WdangTehSeeker3245/belajar-diffusers-stable-diffusion)
* [Precision Info](https://moocaholic.medium.com/fp64-fp32-fp16-bfloat16-tf32-and-other-members-of-the-zoo-a1ca7897d407)

In [ ]:
!pip install git+https://github.com/huggingface/diffusers

In [ ]:
!git clone https://github.com/huggingface/diffusers.git

In [ ]:
!pip install -r /content/diffusers/examples/text_to_image/requirements.txt

In [4]:
!cp /content/diffusers/examples/text_to_image/train_text_to_image_lora.py /content/train_text_to_image_lora.py

In [ ]:
# !pip install xformers

In [ ]:
# !pip install bitsandbytes

Note :    
Fix error in push to hub without validation image

In [ ]:
import re

def modify_code(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Define the regex pattern to find the specific code block to replace
    pattern = re.compile(
        r"( *)(if args\.push_to_hub:\s+)"
        r"(\s*save_model_card\(\s*"
        r"repo_id,\s*"
        r"images=images,\s*"
        r"base_model=args\.pretrained_model_name_or_path,\s*"
        r"dataset_name=args\.dataset_name,\s*"
        r"repo_folder=args\.output_dir,\s*"
        r"\)\s*)"
    )

    # Define the regex pattern to find and delete the previous upload_folder block
    upload_pattern = re.compile(
        r"( *)(upload_folder\(\s*"
        r"repo_id=repo_id,\s*"
        r"folder_path=args\.output_dir,\s*"
        r"commit_message=\"End of training\",\s*"
        r"ignore_patterns=\[\"step_\*\", \"epoch_\*\"\],\s*"
        r"\)\s*)"
        r"""(\s*accelerator\.end_training\(\)\s*)"""
    )

    # Define a function to construct the replacement block with the same indentation
    def replacement(match):
        indent = match.group(1)
        return (
            f"{indent}if args.push_to_hub:\n"
            f"{indent}    if args.validation_prompt is not None:\n"
            f"{indent}        save_model_card(\n"
            f"{indent}            repo_id,\n"
            f"{indent}            images=images,\n"
            f"{indent}            base_model=args.pretrained_model_name_or_path,\n"
            f"{indent}            dataset_name=args.dataset_name,\n"
            f"{indent}            repo_folder=args.output_dir,\n"
            f"{indent}        )\n"
            f"{indent}    else:\n"
            f"{indent}        save_model_card(\n"
            f"{indent}            repo_id,\n"
            f"{indent}            base_model=args.pretrained_model_name_or_path,\n"
            f"{indent}            dataset_name=args.dataset_name,\n"
            f"{indent}            repo_folder=args.output_dir,\n"
            f"{indent}        )\n"
            f"{indent}    upload_folder(\n"
            f"{indent}        repo_id=repo_id,\n"
            f"{indent}        folder_path=args.output_dir,\n"
            f"{indent}        commit_message=\"End of training\",\n"
            f"{indent}        ignore_patterns=[\"step_*\", \"epoch_*\"],\n"
            f"{indent}    )\n"
            f"    accelerator.end_training()\n\n"
        )

    # Remove the previous upload_folder block
    content = upload_pattern.sub("", content)

    # Replace the save_model_card block and add the upload_folder block
    new_content = pattern.sub(replacement, content)

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write(new_content)

# Usage example
file_path = 'train_text_to_image_lora.py'
modify_code(file_path)
print("edit py success")

In [ ]:
!accelerate config default

In [7]:
from huggingface_hub import hf_hub_download,login

In [ ]:
# def read_specific_line(file_path, line_number):
#     try:
#         with open(file_path, 'r') as file:
#             for current_line_number, line in enumerate(file, start=1):
#                 if current_line_number == line_number:
#                     return line.strip()
#         return f"Line {line_number} not found in the file."
#     except FileNotFoundError:
#         return "File not found."
#     except Exception as e:
#         return f"An error occurred: {e}"

# # Example usage
# file_path = "./hf_token.txt"
# line_number = int(input("Enter the line number you want to read: "))
# specific_line = read_specific_line(file_path, line_number)
# print(f"Line {line_number}: {specific_line}")

In [ ]:
# login(token=str(specific_line))

In [ ]:
login(token="your_token")

Test Training Without Checkpointing

In [ ]:
# !accelerate launch train_text_to_image_lora.py \
#   --pretrained_model_name_or_path="ffaizalnf1800/AingDiffusion-V8-FP16-SD1.5-Blessed2Vae" \
#   --dataset_name="finnstrom3693/anime_character_dataset_512" \
#   --resolution=512  \
#   --train_batch_size=4 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=1e-4 \
#   --lr_warmup_steps=5 \
#   --lr_scheduler="cosine_with_restarts" \
#   --num_train_epochs=10 \
#   --output_dir="anime-character-lora-mp-fp16" \
#   --push_to_hub \
#   --mixed_precision="fp16" \
#   --hub_model_id="anime-character-lora-mp-fp16" \
#   --num_validation_images=0 \
#   --adam_weight_decay=1e-2


Training With Checkpointing

In [ ]:
!accelerate launch train_text_to_image_lora.py \
  --pretrained_model_name_or_path="faizalnf1800/AingDiffusion-V8-FP16-SD1.5-Blessed2Vae" \
  --dataset_name="finnstrom3693/anime_character_dataset_512" \
  --resolution=512  \
  --train_batch_size=2 \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-4 \
  --lr_warmup_steps=5 \
  --lr_scheduler="cosine_with_restarts" \
  --num_train_epochs=10 \
  --output_dir="anime-character-lora-mp-fp16" \
  --push_to_hub \
  --mixed_precision="fp16" \
  --hub_model_id="anime-character-lora-mp-fp16" \
  --num_validation_images=0 \
  --adam_weight_decay=1e-2 \
  --checkpointing_steps=250


Full Precision Training With Xformers

In [ ]:
# !accelerate launch train_text_to_image_lora.py \
#   --pretrained_model_name_or_path="faizalnf1800/AingDiffusion-V8-FP16-SD1.5-Blessed2Vae" \
#   --dataset_name="finnstrom3693/anime_character_dataset_512" \
#   --resolution=512  \
#   --train_batch_size=2 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=1e-4 \
#   --lr_warmup_steps=5 \
#   --lr_scheduler="cosine_with_restarts" \
#   --num_train_epochs=10 \
#   --output_dir="anime-character-lora-mp-fp16" \
#   --push_to_hub \
#   --mixed_precision="no" \
#   --enable_xformers_memory_efficient_attention \
#   --hub_model_id="anime-character-lora-mp-fp16" \
#   --num_validation_images=0 \
#   --checkpointing_steps=250 \
#   --adam_weight_decay=1e-2